In [1]:
import pyDes as enc
import re
import num2words
import azure.cognitiveservices.speech as speechsdk
from pydub import AudioSegment
import os
from getpass import getpass

In [2]:
def splitter(n, s):
    pieces = s.split()
    return (" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n))

def ttsRun(text,file_name,voice,test=0):
    if test==0:
        # Creates an instance of a speech config with specified subscription key and service region.
        speech_key = api
        service_region = "canadacentral"

        #voice = "zh-CN-XiaoyiNeural"

        speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
        # Note: the voice setting will not overwrite the voice element in input SSML.
        speech_config.speech_synthesis_voice_name = voice
        speech_config.set_speech_synthesis_output_format(speechsdk.SpeechSynthesisOutputFormat.Audio16Khz32KBitRateMonoMp3)  
  
        file_config = speechsdk.audio.AudioOutputConfig(filename=file_name)
        speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=file_config)
        # Receives a text from console input and synthesizes it to mp3 file.
        result = speech_synthesizer.speak_text_async(text).get()
        # Check result
        if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print("Speech synthesized, and the audio was saved to [{}]".format(file_name))
        elif result.reason == speechsdk.ResultReason.Canceled:
            cancellation_details = result.cancellation_details
            print("Speech synthesis canceled: {}".format(cancellation_details.reason))
            if cancellation_details.reason == speechsdk.CancellationReason.Error:
                print("Error details: {}".format(cancellation_details.error_details))
    
    else:
        # Creates an instance of a speech config with specified subscription key and service region.
        speech_key = api
        service_region = "canadacentral"

        #voice = "zh-CN-XiaoyiNeural"

        speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
        # Note: the voice setting will not overwrite the voice element in input SSML.
        speech_config.speech_synthesis_voice_name = voice
        #reset attributes for testing
        speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
        speech_config.speech_synthesis_voice_name = voice
        # use the default speaker as audio output.
        speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)
        result = speech_synthesizer.speak_text_async(text).get()
        # Check result
        if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print("Speech synthesized for text [{}]".format(text))
        elif result.reason == speechsdk.ResultReason.Canceled:
            cancellation_details = result.cancellation_details
            print("Speech synthesis canceled: {}".format(cancellation_details.reason))
            if cancellation_details.reason == speechsdk.CancellationReason.Error:
                print("Error details: {}".format(cancellation_details.error_details))

In [3]:
enc_api = b'Ox\xc7a\xc22\xc7\x7fOv\xed\x12\x01\x1d\xd175\xcf\xb14\x03F\\\xd6\xf0\xb6\xe9\xd8\xd2ngs\x8b\xc8\\\xd3\x90\xdb\xb9\x11'
key  = getpass()
api = enc.triple_des(key).decrypt(enc_api, padmode=2).decode("utf-8")

········


In [4]:
## Creates an instance of a speech config with specified subscription key and service region.
#speech_key = api
#service_region = "canadacentral"

#voice = "zh-CN-XiaoyiNeural"

#speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
## Note: the voice setting will not overwrite the voice element in input SSML.
#speech_config.speech_synthesis_voice_name = voice

In [5]:
filename = input("filename:")

filename:D:\\AI_stuff\test1.txt


In [6]:
head_tail = os.path.split(filename)

rawName = os.path.splitext(os.path.basename(filename))[0]

if not os.path.exists(head_tail[0]+"\\tmp"):
    # If it doesn't exist, create it
    os.makedirs(head_tail[0]+"\\tmp")
tmpDir = head_tail[0]+"\\tmp"

file_name = head_tail[0] + "\\" + rawName + ".mp3"


In [7]:
with open(filename,"r", encoding="utf8") as f:
    text = f.read()
text = re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), text)

In [8]:
#text = text.replace("Lu Xiaoran","鲁筱冉")

In [10]:
initialVoiceFlag = []
lines = text.split("*")
if (len(lines)>1):
        #voice.append(0)
        for i in range(len(lines)):
            if (i%2) == 0:
                initialVoiceFlag.append(0)
            else:
                initialVoiceFlag.append(1)

finalText = []
voiceFlag = []
position = 0
for string in lines:
    tmpSplit = []
    for split in splitter(1000, string):
        tmpSplit.append(split)
    finalText.extend(tmpSplit)
    tmpVoice = []
    if(position==0):
        tmpVoice.append(0)
    else:
        tmpVoice.append(initialVoiceFlag[position])
    if (len(tmpSplit)>1):
        for i in range(len(tmpSplit)-1):
            tmpVoice.append(0)
    voiceFlag.extend(tmpVoice)
    position +=1


In [14]:
voices = ["zh-CN-XiaoyiNeural"," zh-CN-YunjianNeural"]

In [16]:
outputname = rawName+"_tmp"
partNames = []
for i in range(len(finalText)):
    partNames.append(tmpDir+"\\"+outputname + "part" + str(i+1) + ".mp3")
    vi = voiceFlag[i]
    ttsRun(finalText[i],partNames[i],voices[vi])
print("-----done generating parts-----")

0
Speech synthesized, and the audio was saved to [D:\\AI_stuff\tmp\test1_tmppart1.mp3]
1
Speech synthesized, and the audio was saved to [D:\\AI_stuff\tmp\test1_tmppart2.mp3]
2
Speech synthesized, and the audio was saved to [D:\\AI_stuff\tmp\test1_tmppart3.mp3]
-----done generating parts-----


In [17]:
mp3Parts=[]
for i in range(len(finalText)):
    mp3Parts.append(AudioSegment.from_file(partNames[i], format="mp3"))

print("merging files")    

combinedFile = mp3Parts[0]
for i in range(len(finalText)-1):
    combinedFile = combinedFile + mp3Parts[i+1]

file_handle = combinedFile.export(file_name, format="mp3")
print("writing output to "+file_name)

merging files
writing output to D:\\AI_stuff\test1.mp3
